In [1]:
#conda install -n demvfill ipykernel --update-deps --force-reinstall
from upaths import vpath,rdir 

In [3]:
import os
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
import numpy as np

# Read the vector file
vector = gpd.read_file(vpath)

# Get the bounding box of the vector data
bbox = vector.total_bounds  # [minx, miny, maxx, maxy]

# Set resolution and output CRS
resolution = 1 / 3600  # Set resolution to 1/3600 degree
epsg = 4326  # EPSG:4326

# Calculate the width and height of the raster
width = int((bbox[2] - bbox[0]) / resolution)
height = int((bbox[3] - bbox[1]) / resolution)

# Define the transformation
transform = from_origin(bbox[0], bbox[3], resolution, resolution)

# Function to create a binary raster with random 0s and 1s
def create_random_binary_raster(vpath, rpath):
    # Create a new raster with uint8 data type and high compression
    with rasterio.open(rpath, 'w', driver='GTiff', height=height, width=width,
                       count=1, dtype='uint8', crs=f"EPSG:{epsg}", transform=transform,
                       compress='lzw') as dst:  # 'lzw' is a good compression algorithm for rasters
        # Generate random binary data (0s and 1s)
        random_data = np.random.choice([0, 1], size=(height, width), p=[0.5, 0.5])
        # Write the random data to the raster
        dst.write(random_data.astype('uint8'), 1)

# Call the function to create the binary raster

rpath = vpath.replace(".gpkg", ".tif")
#create_random_binary_raster(vpath, rpath)
os.makedirs(rdir,exist_ok=True)

In [4]:
import subprocess

def tileindex(tdir,gpkg): #pkg = tdir + '.gpkg'
   cmd = f'gdaltindex -t_srs EPSG:4326 -f GPKG {gpkg} {os.path.join(tdir)}/*.tif '
   os.system(cmd)


def gdal_retile_function(input_files, target_dir, pixel_width=256, pixel_height=256, overlap=0, output_format="GTiff"):
    # Construct the command with optional parameters
    os.makedirs(rdir,exist_ok=True)
    command = [
        'gdal_retile', 
        '-ps', str(pixel_width), str(pixel_height),
        #'-levels', str(levels),
        '-overlap', str(overlap),
        '-of', output_format,
        '-targetDir', target_dir
    ] + input_files
    
    # Execute the command
    subprocess.run(command, check=True)

# Example usage:
input_files = [rpath] # List of input files
target_dir = rdir # Output directory for tiles


In [5]:
# gpkg = vpath.replace('.gpkg', '__TIF_TILES.gpkg')
# gdal_retile_function(input_files, target_dir)
# tileindex(tdir=rdir,gpkg=gpkg)

In [ ]:
import ee
import geemap

# Authenticate and initialise Google Earth Engine
ee.Initialize()

# Define Antarctica as the area of interest
geometry = ee.Geometry.Polygon(
    [[[-180, -60], [180, -60], [180, -90], [-180, -90], [-180, -60]]]
)

# Define time range
start_date = '2023-01-01'
end_date = '2023-12-31'

# Load Sentinel-1 data (SAR)
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(geometry) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .select('VV')  # Visualise VV polarisation

# Sentinel-1 Visualisation Parameters
sentinel1_vis_params = {
    'min': -20,
    'max': 0,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# Load Sentinel-2 data (Optical)
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(geometry) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .select(['B4', 'B3', 'B2'])  # True colour (RGB)

# Sentinel-2 Visualisation Parameters
sentinel2_vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

# Create a map with geemap
Map = geemap.Map(center=[-75, 0], zoom=3)  # Centred over Antarctica

# Add Sentinel-1 and Sentinel-2 layers to the map
Map.addLayer(sentinel1.median().clip(geometry), sentinel1_vis_params, 'Sentinel-1 (SAR)')
Map.addLayer(sentinel2.median().clip(geometry), sentinel2_vis_params, 'Sentinel-2 (Optical)')

# Add layer control and display the map
Map.addLayerControl()
Map


In [ ]:
import ee
import geemap

# Initialise the Earth Engine library
ee.Initialize()

# --- Mask Edge Function ---
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

# --- Sentinel-1 (VV Polarisation) Analysis ---
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# Define seasonal time filters
spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

# Create multi-temporal composites for descending and ascending passes
desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

# --- Sentinel-2 Cloud Masking ---
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = (
        qa.bitwiseAnd(cloud_bit_mask).eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )
    return image.updateMask(mask).divide(10000)

dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-01-30')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

# Sentinel-2 RGB visualisation parameters
rgb_vis = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}

# --- Map Visualisation ---
m = geemap.Map()
m.set_center(83.277, 17.7009, 12)  # Example coordinates
m.add_layer(dataset.mean(), rgb_vis, 'Sentinel-2 RGB')
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m

# --- Dynamic World and Sentinel-2 Comparison ---
start = ee.Date('2021-04-02')
end = start.advance(1, 'day')
col_filter = ee.Filter.And(
    ee.Filter.bounds(ee.Geometry.Point(20.6729, 52.4305)),
    ee.Filter.date(start, end),
)

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
linked_col = dw_col.linkCollection(s2_col, s2_col.first().bandNames())
linked_image = ee.Image(linked_col.first())

# Dynamic World Visualisation Parameters
class_names = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice',
]
vis_palette = [
    '419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a',
    'c4281b', 'a59b8f', 'b39fe1',
]
dw_rgb = (
    linked_image.select('label')
    .visualize(min=0, max=8, palette=vis_palette)
    .divide(255)
)
top1_prob = linked_image.select(class_names).reduce(ee.Reducer.max())
top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)
dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)

m = geemap.Map()
m.set_center(20.6729, 52.4305, 12)
m.add_layer(
    linked_image,
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 L1C',
)
m.add_layer(
    dw_rgb_hillshade,
    {'min': 0, 'max': 0.65},
    'Dynamic World V1 - label hillshade',
)
m

# --- ESA WorldCover Dataset ---
world_cover = ee.ImageCollection('ESA/WorldCover/v100').first()
m = geemap.Map()
m.center_object(world_cover)
m.add_layer(world_cover, {'bands': ['Map']}, 'Landcover')
m


# viz all the dataset only in antartica , including st george island

In [10]:
import ee
import geemap

# Initialise the Earth Engine library
ee.Initialize()

# Define the AOI for Antarctica (including St. George Island)
aoi = ee.Geometry.Rectangle([-60.0, -63.0, -56.0, -61.0])  # Adjust bounding box as needed

# --- Sentinel-1 (VV Polarisation) Analysis for Antarctica ---
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filterBounds(aoi)  # Filter to Antarctica AOI
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# Define seasonal time filters
spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

# Create multi-temporal composites
desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

# --- Sentinel-2 Cloud Masking in Antarctica ---
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = (
        qa.bitwiseAnd(cloud_bit_mask).eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )
    return image.updateMask(mask).divide(10000)

s2_dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(aoi)
    .filterDate('2020-01-01', '2020-01-30')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

# --- Dynamic World and Sentinel-2 Comparison ---
start = ee.Date('2021-04-02')
end = start.advance(1, 'day')
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(aoi).filterDate(start, end)
s2_col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(aoi)

linked_col = dw_col.linkCollection(s2_col, s2_col.first().bandNames())
linked_image = ee.Image(linked_col.first())

# --- ESA WorldCover Dataset for Antarctica ---
world_cover = ee.ImageCollection('ESA/WorldCover/v100').first().clip(aoi)

# --- Visualisation Parameters ---
s1_vis = {'min': -25, 'max': 5}
s2_vis = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}
dw_vis = {'min': 0, 'max': 0.65}
world_cover_vis = {'bands': ['Map']}

# --- Map Visualisation ---
m = geemap.Map()
m.center_object(aoi, 5)  # Zoom to Antarctica AOI
m.add_layer(s2_dataset.mean(), s2_vis, 'Sentinel-2 RGB')
m.add_layer(asc_change, s1_vis, 'Sentinel-1 ASC')
m.add_layer(desc_change, s1_vis, 'Sentinel-1 DESC')
#m.add_layer(linked_image, {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}, 'Sentinel-2 L1C')
m.add_layer(world_cover, world_cover_vis, 'WorldCover')
m


Map(center=[-62.003523740394606, -57.99999999999998], controls=(WidgetControl(options=['position', 'transparen…

In [12]:
import ee
import geemap

# Initialise the Earth Engine library
ee.Initialize()

# Define AOI for King George Island
king_george_aoi = ee.Geometry.Rectangle([-59.5, -62.3, -57.5, -61.8])  # Bounding box for King George Island

# --- Sentinel-1 (VV Polarisation) Analysis for King George Island ---
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filterBounds(king_george_aoi)
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# Define seasonal time filters
spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

# Create multi-temporal composites
desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

# --- Sentinel-2 Cloud Masking for King George Island ---
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = (
        qa.bitwiseAnd(cloud_bit_mask).eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )
    return image.updateMask(mask).divide(10000)

s2_dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(king_george_aoi)
    .filterDate('2020-01-01', '2020-01-30')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

# --- Dynamic World and Sentinel-2 Comparison ---
start = ee.Date('2021-04-02')
end = start.advance(1, 'day')
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(king_george_aoi).filterDate(start, end)
s2_col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(king_george_aoi)

linked_col = dw_col.linkCollection(s2_col, s2_col.first().bandNames())
linked_image = ee.Image(linked_col.first())

# --- ESA WorldCover Dataset for King George Island ---
world_cover = ee.ImageCollection('ESA/WorldCover/v100').first().clip(king_george_aoi)

# --- Visualisation Parameters ---
s1_vis = {'min': -25, 'max': 5}
s2_vis = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}
dw_vis = {'min': 0, 'max': 0.65}
world_cover_vis = {'bands': ['Map']}

# --- Map Visualisation ---
m = geemap.Map()
m.center_object(king_george_aoi, 10)  # Zoom to King George Island
m.add_layer(s2_dataset.mean(), s2_vis, 'Sentinel-2 RGB')
m.add_layer(asc_change, s1_vis, 'Sentinel-1 ASC')
m.add_layer(desc_change, s1_vis, 'Sentinel-1 DESC')
#m.add_layer(linked_image, {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}, 'Sentinel-2 L1C')
m.add_layer(world_cover, world_cover_vis, 'WorldCover')
m


Map(center=[-62.0529277650045, -58.50000000000013], controls=(WidgetControl(options=['position', 'transparent_…

In [18]:
import ee
import geemap

# Initialise the Earth Engine library
ee.Initialize()

# Define AOI for King George Island
king_george_aoi = ee.Geometry.Rectangle([-59.5, -62.3, -57.5, -61.8])  # Bounding box for King George Island

# --- Sentinel-1 (VV Polarisation) Analysis ---
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)


di = '2015-01-01'
df = '2023-12-30'

img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filterBounds(king_george_aoi)
    .filterDate(di, df)
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# Print the number of images in Sentinel-1 collections
print("Total Sentinel-1 Images:", img_vv.size().getInfo())
print("Descending Orbit Images:", desc.size().getInfo())
print("Ascending Orbit Images:", asc.size().getInfo())

# --- Sentinel-2 Cloud Masking ---
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = (
        qa.bitwiseAnd(cloud_bit_mask).eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )
    return image.updateMask(mask).divide(10000)



s2_dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(king_george_aoi)
    .filterDate(di, df)
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

# Print the number of images in Sentinel-2 collection
print("Total Sentinel-2 Images:", s2_dataset.size().getInfo())


# --- Dynamic World Dataset ---
start = ee.Date(di)
end = ee.Date(df)#start.advance(1000, 'day')
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(king_george_aoi).filterDate(start, end)

# Print the number of images in Dynamic World collection
print("Total Dynamic World Images:", dw_col.size().getInfo())

# --- ESA WorldCover Dataset ---
world_cover = ee.ImageCollection('ESA/WorldCover/v100').first().clip(king_george_aoi)

# --- Visualisation Parameters ---
s1_vis = {'min': -25, 'max': 5}
s2_vis = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}
world_cover_vis = {'bands': ['Map']}

# --- Map Visualisation ---
m = geemap.Map()
m.center_object(king_george_aoi, 10)  # Zoom to King George Island
m.add_layer(s2_dataset.median().clip(king_george_aoi), s2_vis, 'Sentinel-2 RGB')
m.add_layer(asc.median().clip(king_george_aoi), s1_vis, 'Sentinel-1 ASC')
m.add_layer(desc.median().clip(king_george_aoi), s1_vis, 'Sentinel-1 DESC')
m.add_layer(world_cover.clip(king_george_aoi), world_cover_vis, 'WorldCover')
m.add_layer(king_george_aoi,{}, 'ROI') # make this transparent, and just show the outline
# use the other package 

Total Sentinel-1 Images: 0
Descending Orbit Images: 0
Ascending Orbit Images: 0
Total Sentinel-2 Images: 1535
Total Dynamic World Images: 356


In [19]:
m

Map(center=[-62.0529277650045, -58.50000000000013], controls=(WidgetControl(options=['position', 'transparent_…

In [22]:
import ee
import geemap

# Initialise the Earth Engine library
ee.Initialize()

# Define AOI for King George Island
king_george_aoi = ee.Geometry.Rectangle([-59.5, -62.3, -57.5, -61.8])  # Bounding box for King George Island

# --- Sentinel-1 (VV Polarisation) Analysis ---
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

di = '2012-04-02'
df = '2023-01-30'

img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filterBounds(king_george_aoi)
    .filterDate(di, df)
    #.select('VV')
    #.map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# Print the number of images in Sentinel-1 collections
print("Total Sentinel-1 Images:", img_vv.size().getInfo())
print("Descending Orbit Images:", desc.size().getInfo())
print("Ascending Orbit Images:", asc.size().getInfo())

# --- Sentinel-2 Cloud Masking ---
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = (
        qa.bitwiseAnd(cloud_bit_mask).eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )
    return image.updateMask(mask).divide(10000)

# Relaxing the cloudy pixel filter (try removing the filter temporarily)
s2_dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(king_george_aoi)
    .filterDate(di, df)
    .map(mask_s2_clouds)
)

# Print the number of images in Sentinel-2 collection
print("Total Sentinel-2 Images:", s2_dataset.size().getInfo())

# --- Dynamic World Dataset ---
start = ee.Date(di)
end = start.advance(30, 'day')  # Increasing the range from a day to a 30-day window
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(king_george_aoi).filterDate(start, end)

# Print the number of images in Dynamic World collection
print("Total Dynamic World Images:", dw_col.size().getInfo())

# --- ESA WorldCover Dataset ---
world_cover = ee.ImageCollection('ESA/WorldCover/v100').first().clip(king_george_aoi)

# --- Visualisation Parameters ---
s1_vis = {'min': -25, 'max': 5}
s2_vis = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}
world_cover_vis = {'bands': ['Map']}

# --- Map Visualisation ---
m = geemap.Map()
m.center_object(king_george_aoi, 10)  # Zoom to King George Island
m.add_layer(s2_dataset.median().clip(king_george_aoi), s2_vis, 'Sentinel-2 RGB')
m.add_layer(asc.median().clip(king_george_aoi), s1_vis, 'Sentinel-1 ASC')
m.add_layer(desc.median().clip(king_george_aoi), s1_vis, 'Sentinel-1 DESC')
m.add_layer(world_cover.clip(king_george_aoi), world_cover_vis, 'WorldCover')
m.add_layer(king_george_aoi, {}, 'ROI')  # Outline of AOI
m


Total Sentinel-1 Images: 0
Descending Orbit Images: 0
Ascending Orbit Images: 0
Total Sentinel-2 Images: 1093
Total Dynamic World Images: 0


Map(center=[-62.0529277650045, -58.50000000000013], controls=(WidgetControl(options=['position', 'transparent_…

In [25]:
# too much cloud + less coverage : use other globally available datasets:
# - egm08, egm08, others egms, bathy, dems, modis (250) nvdi and other stuff 